In [75]:
import logging
import os
import sys
import numpy as np
import pandas as pd
import torch
from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.handlers import EarlyStopping, ModelCheckpoint
from ignite.metrics import Accuracy

import monai
import tensorboard
from monai.data import ImageDataset, decollate_batch, DataLoader
from monai.handlers import StatsHandler, TensorBoardStatsHandler, stopping_fn_from_metric
from monai.transforms import EnsureChannelFirst, Compose, RandRotate90, Resize, ScaleIntensity

monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 1.3.0
Numpy version: 1.24.3
Pytorch version: 2.1.0+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: c:\Users\<username>\anaconda3\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.12
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.20.0
scipy version: 1.11.1
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: 4.7.1
TorchVision version: 0.16.0+cpu
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.0
pandas version: 2.0.3
einops version: 0.7.0
transformers version: 4.32.1
mlflow version: 2.7.1
pynrrd version: 1.0.0
clearml version: 1.13.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [8]:
#list of image paths
imgFolder = './LiverSegmentations'
images = []
for folders in os.listdir(imgFolder):
    for files in os.listdir(os.path.join(imgFolder, folders)):
        imgPath = os.path.join(imgFolder, folders, files)
        images.append(imgPath)

In [10]:
#extract labels
labelPath = "./Colorectal Liver Metastases Clinical data April 2023.xlsx"
data = pd.read_excel(labelPath, sheet_name="CRLM (n=168)")
rec_5yr = data['Recurrence (5y)']
rec_2yr = data['Recurrence (2y)']
data_id = data['Patient-ID']
data_label = pd.concat([data_id, rec_2yr, rec_5yr], axis=1)

In [11]:
print(data_label.head())

     Patient-ID  Recurrence (2y)  Recurrence (5y)
0  CRLM-CT-1001                0                0
1  CRLM-CT-1002                1                1
2  CRLM-CT-1003                0                0
3  CRLM-CT-1004                0                0
4  CRLM-CT-1005                0                1


In [12]:
images = sorted(images)

In [13]:
paths = []
for patients in data_label['Patient-ID']:
    for image in images:
        if image.find(patients) != -1:
            paths.append(image)

path_toadd = {'Path':paths}
paths_df = pd.DataFrame(path_toadd)
data_label = pd.concat([data_label, paths_df], axis=1)

In [14]:
labels_5yr = np.array(data_label['Recurrence (5y)'], dtype=np.int64)
labels_2yr = np.array(data_label['Recurrence (2y)'], dtype=np.int64)

In [15]:
from sklearn.model_selection import train_test_split

x_train2, x_val2, y_train2, y_val2 = train_test_split(data_label, labels_2yr, test_size=0.3)

In [16]:
print(len(x_train2))
print(len(x_val2))

117
51


In [17]:
import nibabel as nib
import matplotlib.pyplot as plt
min_z = 10000
for paths in data_label['Path']:
    test_load = nib.load(paths).get_fdata()
    if test_load.shape[2] < min_z:
        min_z = test_load.shape[2]

print(min_z)

15


In [55]:
# define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((256, 256, 48)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((256, 256, 48))])

In [56]:
# define image dataset, data loader
check_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label)

<class 'monai.data.meta_tensor.MetaTensor'> torch.Size([2, 1, 256, 256, 48]) tensor([0, 1])


In [77]:
# create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
net = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss = torch.nn.CrossEntropyLoss()
lr = 1e-5
opt = torch.optim.Adam(net.parameters(), lr)

cpu


In [76]:
print(torch.backends.cudnn.enabled)
print(torch.cuda.is_available())
print(torch.__version__)

True
False
2.1.0+cpu


In [58]:
# Ignite trainer expects batch=(img, label) and returns output=loss at every iteration,
# user can add output_transform to return other values, like: y_pred, y, etc.
trainer = create_supervised_trainer(net, opt, loss, device, False)

In [59]:

# adding checkpoint handler to save models (network params and optimizer stats) during training
checkpoint_handler = ModelCheckpoint("./runs_array/", "net", n_saved=10, require_empty=False)
trainer.add_event_handler(
    event_name=Events.EPOCH_COMPLETED, handler=checkpoint_handler, to_save={"net": net, "opt": opt}
)

In [60]:
# StatsHandler prints loss at every iteration and print metrics at every epoch,
# we don't set metrics for trainer here, so just print loss, user can also customize print functions
# and can use output_transform to convert engine.state.output if it's not loss value
train_stats_handler = StatsHandler(name="trainer", output_transform=lambda x: x)
train_stats_handler.attach(trainer)

In [61]:
# TensorBoardStatsHandler plots loss at every iteration and plots metrics at every epoch, same as StatsHandler
train_tensorboard_stats_handler = TensorBoardStatsHandler(output_transform=lambda x: x)
train_tensorboard_stats_handler.attach(trainer)

In [62]:
# set parameters for validation
validation_every_n_epochs = 1

In [63]:
metric_name = "Accuracy"
# add evaluation metric to the evaluator engine
val_metrics = {metric_name: Accuracy()}
# Ignite evaluator expects batch=(img, label) and returns output=(y_pred, y) at every iteration,
# user can add output_transform to return other values
evaluator = create_supervised_evaluator(net, val_metrics, device, True)


In [64]:
# add stats event handler to print validation stats via evaluator
val_stats_handler = StatsHandler(
    name="evaluator",
    output_transform=lambda x: None,  # no need to print loss value, so disable per iteration output
    global_epoch_transform=lambda x: trainer.state.epoch,
)  # fetch global epoch number from trainer
val_stats_handler.attach(evaluator)

In [65]:
# add early stopping handler to evaluator
early_stopper = EarlyStopping(patience=4, score_function=stopping_fn_from_metric(metric_name), trainer=trainer)
evaluator.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=early_stopper)

In [66]:
# create a validation data loader
val_ds = ImageDataset(image_files=np.array(x_val2['Path']), labels=np.array(y_val2), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())

In [67]:
@trainer.on(Events.EPOCH_COMPLETED(every=validation_every_n_epochs))
def run_validation(engine):
    evaluator.run(val_loader)

In [68]:
# create a training data loader
train_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

In [69]:
train_epochs = 30
state = trainer.run(train_loader, train_epochs)
print(state)

INFO:ignite.engine.engine.Engine:Engine run starting with max_epochs=30.
2023-10-13 15:22:57,420 - INFO - Epoch: 1/30, Iter: 1/59 -- Loss: 0.6764 
2023-10-13 15:23:02,414 - INFO - Epoch: 1/30, Iter: 2/59 -- Loss: 0.6644 
2023-10-13 15:23:06,811 - INFO - Epoch: 1/30, Iter: 3/59 -- Loss: 0.7566 
2023-10-13 15:23:11,114 - INFO - Epoch: 1/30, Iter: 4/59 -- Loss: 0.6979 
2023-10-13 15:23:15,402 - INFO - Epoch: 1/30, Iter: 5/59 -- Loss: 0.7194 
2023-10-13 15:23:19,730 - INFO - Epoch: 1/30, Iter: 6/59 -- Loss: 0.8235 
2023-10-13 15:23:24,010 - INFO - Epoch: 1/30, Iter: 7/59 -- Loss: 0.8292 
2023-10-13 15:23:28,262 - INFO - Epoch: 1/30, Iter: 8/59 -- Loss: 0.6330 
2023-10-13 15:23:32,550 - INFO - Epoch: 1/30, Iter: 9/59 -- Loss: 0.5410 
2023-10-13 15:23:36,835 - INFO - Epoch: 1/30, Iter: 10/59 -- Loss: 0.5989 
2023-10-13 15:23:41,147 - INFO - Epoch: 1/30, Iter: 11/59 -- Loss: 0.6821 
2023-10-13 15:23:45,464 - INFO - Epoch: 1/30, Iter: 12/59 -- Loss: 0.8125 
2023-10-13 15:23:49,709 - INFO - Epo

2023-10-13 15:46:39,970 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


INFO:ignite.engine.engine.Engine:Terminate signaled. Engine will stop after current iteration is finished.
INFO:ignite.engine.engine.Engine:Epoch[1] Complete. Time taken: 00:00:23.103
INFO:ignite.engine.engine.Engine:Engine run complete. Time taken: 00:00:23.207
INFO:ignite.engine.engine.Engine:Engine run complete. Time taken: 00:23:52.304
State:
	iteration: 295
	epoch: 5
	epoch_length: 59
	max_epochs: 30
	output: 1.157374382019043
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'monai.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

